In [ ]:
import speech_recognition as sr
import RPi.GPIO as GPIO
import time
import os
import json
from gpiozero import LED  # Alternative to RPi.GPIO

class VoiceControlledAppliances:
    def __init__(self):
        # Initialize GPIO (Raspberry Pi)
        GPIO.setmode(GPIO.BCM)
        self.appliances = {
            "living room light": LED(17),
            "kitchen light": LED(18),
            "fan": LED(22)
        }
        
        # Initialize speech recognizer
        self.recognizer = sr.Recognizer()
        self.recognizer.energy_threshold = 4000
        self.recognizer.dynamic_energy_threshold = True
        
        # Load configuration
        self.load_config()
        
        # Wake word detection (simple version)
        self.wake_word = "jarvis"
        
    def load_config(self):
        """Load appliance configuration from JSON file"""
        try:
            with open('appliances_config.json') as f:
                config = json.load(f)
                for name, pin in config.items():
                    self.appliances[name] = LED(pin)
        except FileNotFoundError:
            print("Configuration file not found. Using default setup.")
        except Exception as e:
            print(f"Error loading config: {e}")

    def listen_for_command(self):
        """Listen to microphone and return recognized text"""
        with sr.Microphone() as source:
            print("Adjusting for ambient noise...")
            self.recognizer.adjust_for_ambient_noise(source, duration=1)
            print("Listening for command...")
            audio = self.recognizer.listen(source, timeout=5, phrase_time_limit=5)
            
        try:
            command = self.recognizer.recognize_google(audio).lower()
            print(f"You said: {command}")
            return command
        except sr.UnknownValueError:
            print("Sorry, I didn't understand that.")
            return ""
        except sr.RequestError:
            print("Could not request results from Google Speech Recognition.")
            return ""
        except sr.WaitTimeoutError:
            print("Listening timed out.")
            return ""

    def process_command(self, command):
        """Process voice command and control appliances"""
        if not command:
            return
            
        # Check for wake word
        if self.wake_word not in command:
            return
            
        # Remove wake word from command
        command = command.replace(self.wake_word, "").strip()
        
        # Simple command processing
        action = None
        appliance = None
        
        # Determine action
        if "turn on" in command:
            action = "on"
            command = command.replace("turn on", "")
        elif "turn off" in command:
            action = "off"
            command = command.replace("turn off", "")
        elif "switch on" in command:
            action = "on"
            command = command.replace("switch on", "")
        elif "switch off" in command:
            action = "off"
            command = command.replace("switch off", "")
            
        # Find appliance in command
        for app_name in self.appliances.keys():
            if app_name in command:
                appliance = app_name
                break
                
        # Execute command if valid
        if action and appliance:
            self.control_appliance(appliance, action)
        else:
            print("Command not recognized. Try something like 'turn on living room light'")

    def control_appliance(self, appliance_name, action):
        """Control the specified appliance"""
        if appliance_name not in self.appliances:
            print(f"Unknown appliance: {appliance_name}")
            return
            
        appliance = self.appliances[appliance_name]
        
        if action == "on":
            appliance.on()
            print(f"{appliance_name} turned ON")
        elif action == "off":
            appliance.off()
            print(f"{appliance_name} turned OFF")
        else:
            print(f"Unknown action: {action}")

    def run(self):
        """Main execution loop"""
        print("Voice controlled appliances system ready")
        print(f"Say '{self.wake_word}' followed by your command")
        
        try:
            while True:
                command = self.listen_for_command()
                self.process_command(command)
                time.sleep(0.1)
        except KeyboardInterrupt:
            print("\nShutting down...")
            GPIO.cleanup()

if __name__ == "__main__":
    vca = VoiceControlledAppliances()
    vca.run()